<a href="https://colab.research.google.com/github/ameyas1/Blackpink_Face_Recognition/blob/master/blackpink_video.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from tensorflow.keras.applications.vgg16 import preprocess_input
from tensorflow.keras.preprocessing.image import load_img,img_to_array
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

%matplotlib inline

In [0]:
from urllib.request import urlopen
import os
import bz2
import matplotlib.patches as patches

from align import AlignDlib


def download_landmarks(dst_file):
    url = 'http://dlib.net/files/shape_predictor_68_face_landmarks.dat.bz2'
    decompressor = bz2.BZ2Decompressor()

    with urlopen(url) as src, open(dst_file, 'wb') as dst:
        data = src.read(1024)
        while len(data) > 0:
            dst.write(decompressor.decompress(data))
            data = src.read(1024)


dst_dir = 'model1'
dst_file = os.path.join(dst_dir, 'landmarks.dat')

if not os.path.exists(dst_file):
    os.makedirs(dst_dir)
    download_landmarks(dst_file)

In [0]:
# Initialize the OpenFace face alignment utility
alignment = AlignDlib('model1/landmarks.dat')

In [0]:

import cv2
def load_image(path):
    img = cv2.imread(path, 1)
    # OpenCV loads images with color channels
    # in BGR order. So we need to reverse them
#     b,g,r = cv2.split(img)     
#     img = cv2.merge([r,g,b])
#     return img
    return img[...,::-1]

In [0]:
id_dict={0:'Jennie',1:'Jisoo',2:'Lisa',3:'Rose'}

In [0]:
def prediction(img_path,target):
  img = load_image(img_path)
  
#   img_aligned = alignment.align(224, img, bb, landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)
  bb = alignment.getAllFaceBoundingBoxes(img)
  plt.figure(figsize=(15,15))
  plt.imshow(img)
  for i in bb:
    plt.gca().add_patch(patches.Rectangle((i.left(), i.top()), i.width(), i.height(), fill=False, color='red'))
    img_aligned = alignment.align(224, img, i, landmarkIndices=AlignDlib.OUTER_EYES_AND_NOSE)
    x=img_to_array(img_aligned)
    x = preprocess_input(x)
    x=np.expand_dims(x, axis=0)
    classes = fc_model.predict(x, batch_size=32)[0]
    id=np.argmax(classes)
    for j in id_dict:
      if j==id:
        identity=id_dict[id]
    plt.text(i.left() - 10, i.top() - 10,'{}'.format(identity), color='red',fontsize=17)
  plt.savefig(target)
  plt.close()

In [8]:
from tensorflow.keras.models import load_model
fc_model=load_model('drive/My Drive/cp/weights-184-0.94.hdf5')

W0625 16:19:01.490476 140069894027136 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling GlorotUniform.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0625 16:19:01.496619 140069894027136 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
W0625 16:19:01.500273 140069894027136 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:97: calling Zeros.__init__ (from tensorflow.python.ops.init_ops) with dtyp

In [0]:
!mkdir vid2img

In [0]:
!mkdir temp

In [0]:
import cv2
img_array = []

 # Opens the Video file
cap= cv2.VideoCapture('blackpink_video_input.mp4')
i=0
while(cap.isOpened()):
    ret, frame = cap.read()
    if ret == False:
        break
    cv2.imwrite('vid2img/ktl'+str(i)+'.jpg',frame)
    prediction('vid2img/ktl'+str(i)+'.jpg','temp/temp.jpg')
    img = cv2.imread('temp/temp.jpg')
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
    i+=1
 
cap.release()
cv2.destroyAllWindows()

out = cv2.VideoWriter('blackpink.avi',cv2.VideoWriter_fourcc(*'DIVX'), 23, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()